In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
c_folder="../../data/calibration/"
d_folder="../../data/runData/"
r_n=307
p_folder="../../plots/run_"+str(r_n)+"/"

In [4]:
# helper functions
def choose_bar(df,layer, strip): #works
    df=df[df["strip"]==strip]
    df=df[df["layer"]==layer]
    return df

def import_data(calibration_folder, data_folder, run_number, is_it_pulsed=False): # works for both pulsed and not-pulsed
    
    pedestal_df=pd.read_csv(calibration_folder+"pedestal_fixed.csv", sep=',')
    mip_df=pd.read_csv(calibration_folder+"mip.csv", sep=',')
    
    if is_it_pulsed==True:
        run_df=pd.read_csv(data_folder+"run_"+str(run_number)+"_pulse.csv", sep=',')
        run_df.drop(["pf_spill", "pf_ticks"], axis=1, inplace=True) # we never need these
        run_df.drop(["toa_end0", "toa_end1"], axis=1, inplace=True) # unreliable guys, but we might need them in the future
    else:
        run_df=pd.read_csv(data_folder+"run_"+str(run_number)+".csv", sep=',')
        run_df.drop(["pf_spill", "pf_ticks"], axis=1, inplace=True)
        run_df.drop(["toa_end0", "toa_end1"], axis=1, inplace=True)
        
    return pedestal_df,mip_df,run_df

In [33]:
def drop_extra_pedestal_data(df):
    df.drop(["pedestal_end0", "pedestal_end1",
             "pedestal_per_time_sample_end0","pedestal_per_time_sample_end1"], axis=1, inplace=True)
    return df

In [8]:
def subtract_pedestals(df, is_it_pulsed=False):
    df["adc_sum_end0"]-=df["pedestal_end0"]
    df["adc_sum_end1"]-=df["pedestal_end1"]
    
    df["adc_max_end0"]-=df["pedestal_per_time_sample_end0"]
    df["adc_max_end1"]-=df["pedestal_per_time_sample_end1"]
    
    df["adc_mean_end0"]-=df["pedestal_per_time_sample_end0"]
    df["adc_mean_end1"]-=df["pedestal_per_time_sample_end1"]

    if is_it_pulsed==True:
        for i in range(8):
            adc_end0_col = f'adc_{i}_end0'
            adc_end1_col = f'adc_{i}_end1'
        
            df[adc_end0_col]-=df["pedestal_per_time_sample_end0"]
            df[adc_end1_col]-=df["pedestal_per_time_sample_end1"]
    
    return df

In [34]:
def select_bars_with_data(run_df, pedestal_df, mip_df=None, subtract_pedestal=False, convert_mip=False, is_it_pulsed=False):
    
    merged_df=run_df.merge(pedestal_df, how='left', on=['layer', 'strip'])

    merged_df=merged_df[(merged_df["adc_sum_end0"]>(merged_df["pedestal_end0"]+5*merged_df["std_dev_end0"])) & 
                        (merged_df["adc_sum_end1"]>(merged_df["pedestal_end1"]+5*merged_df["std_dev_end1"]))]
    
    merged_df.drop(["std_dev_end0", "std_dev_end1"], axis=1, inplace=True)

    if subtract_pedestal==True:
        merged_df=subtract_pedestals(merged_df, is_it_pulsed)

    merged_df=drop_extra_pedestal_data(merged_df) # needs to be dropped regardless of subtraction, too much memory used otherwise

    if convert_mip==True: #similarily
        re_merged_df=merged_df.merge(mip_df)
        
    return merged_df

In [30]:
p,m,r=import_data(c_folder,d_folder, 300, is_it_pulsed=True)

In [31]:
r=select_bars_with_data(r,p, subtract_pedestal=True, is_it_pulsed=True) #this needs to be run as it's own cell

In [32]:
r.sample(10)

,pf_event,tot_end0,adc_sum_end0,adc_mean_end0,adc_max_end0,adc_0_end0,adc_1_end0,adc_2_end0,adc_3_end0,adc_4_end0,adc_5_end0,adc_6_end0,adc_7_end0,layer,strip,tot_end1,adc_sum_end1,adc_mean_end1,adc_max_end1,adc_0_end1,adc_1_end1,adc_2_end1,adc_3_end1,adc_4_end1,adc_5_end1,adc_6_end1,adc_7_end1,pedestal_per_time_sample_std_dev_end0,pedestal_per_time_sample_std_dev_end1
1438763,11584,0,392,49.000,94,0,-30,2,56,90,94,86,94,3,4,0,449,56.125,98,-3,-25,23,76,97,96,87,98,1.834212,2.410156
2612838,46452,0,78,9.750,54,-26,-20,4,25,54,33,10,-2,5,4,0,98,12.250,53,-26,-16,10,31,53,34,10,2,2.060949,1.872368
4795228,11497,0,694,86.750,244,10,49,82,244,144,75,50,40,9,3,0,485,60.625,172,4,26,49,172,118,58,34,24,1.360501,2.756437
2060114,66586,217,3676,459.500,922,46,922,922,922,410,410,66,-22,4,4,2904,3328,416.000,928,36,928,928,928,416,18,76,-2,1.924747,1.077873
905676,53938,0,62,7.750,25,-3,-16,-1,13,18,25,16,10,2,4,0,53,6.625,23,-4,-14,-2,12,16,23,14,8,1.653789,1.017445
3103216,35602,1864,5807,725.875,928,228,928,928,928,928,928,928,11,6,3,353,4789,598.625,928,146,928,928,928,928,928,52,-49,1.312134,1.197089
845354,65851,0,7328,916.000,916,916,916,916,916,916,916,916,916,2,3,0,7328,916.000,916,916,916,916,916,916,916,916,916,1.861981,1.484994
2462974,38426,0,157,19.625,63,-4,-26,-9,24,48,63,41,20,5,2,0,164,20.500,58,-7,-26,-6,28,47,58,42,28,1.220176,2.132965
3059808,66456,0,85,10.625,35,-2,-30,-9,20,35,31,32,8,6,2,0,109,13.625,38,2,-30,-8,22,38,36,36,13,1.859809,1.991965
2606765,39586,0,432,54.000,135,0,-34,25,68,84,135,94,60,5,4,0,349,43.625,86,-2,-34,16,56,77,86,84,66,2.060949,1.872368
